# Volume Rata of Change

In [2]:
def calculate_vroc(volume_data, n):
    """
    Calculate the Volume Rate of Change (VROC)
    
    Parameters:
    volume_data (list or array-like): A list or array of volume data, with the most recent volume at the end.
    n (int): The number of periods over which to calculate the VROC.

    Returns:
    float: The Volume Rate of Change (VROC) value.
    """
    if len(volume_data) < n + 1:
        raise ValueError("Not enough data to calculate VROC for the given period n.")

    V_t = volume_data[-1]
    
    V_tn = volume_data[-n-1]
    
    vroc = ((V_t - V_tn) / V_tn) * 100
    
    return vroc

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('data/zeel/ZEEL (20240930093400000 _ 20240925105700000).csv')
data

,Date,Open,High,Low,Close,Jaw ‌Alligator‌,Teeth ‌Alligator,Lips ‌Alligator,MA5,MA15,Volume,Volume Change Percentage
0,Wed Sep 25 2024 10:57:00 GMT+0530 (India Stand...,128.35,128.36,128.27,128.34,127.57,127.87,128.47,128.42,128.54,8697.0,-68.27
1,Wed Sep 25 2024 10:58:00 GMT+0530 (India Stand...,128.34,128.40,128.32,128.39,127.58,127.96,128.44,128.38,128.55,18923.0,-85.49
2,Wed Sep 25 2024 10:59:00 GMT+0530 (India Stand...,128.37,128.39,128.30,128.34,127.60,128.06,128.42,128.36,128.54,17725.0,-63.08
3,Wed Sep 25 2024 11:00:00 GMT+0530 (India Stand...,128.34,128.35,128.30,128.35,127.61,128.14,128.41,128.35,128.52,4863.0,-95.12
4,Wed Sep 25 2024 11:01:00 GMT+0530 (India Stand...,128.35,128.40,128.30,128.35,127.65,128.18,128.39,128.35,128.49,10833.0,-74.49
...,...,...,...,...,...,...,...,...,...,...,...,...
1018,Fri Sep 27 2024 15:25:00 GMT+0530 (India Stand...,135.14,135.38,135.05,135.17,135.74,135.88,135.50,135.27,135.70,96944.0,-58.69
1019,Fri Sep 27 2024 15:26:00 GMT+0530 (India Stand...,135.15,135.37,135.10,135.10,135.75,135.88,135.45,135.23,135.64,123132.0,-33.40
1020,Fri Sep 27 2024 15:27:00 GMT+0530 (India Stand...,135.15,135.30,135.06,135.30,135.76,135.88,135.40,135.23,135.61,119850.0,-36.70
1021,Fri Sep 27 2024 15:28:00 GMT+0530 (India Stand...,135.24,135.30,135.05,135.10,135.77,135.87,135.36,135.17,135.55,122398.0,-57.00


In [5]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Jaw ‌Alligator‌',
       'Teeth ‌Alligator', 'Lips ‌Alligator', 'MA5', 'MA15', 'Volume',
       'Volume Change Percentage'],
      dtype='object')

In [6]:
data = data.to_numpy()

In [7]:
last_14_volume = data[:15, 10]
last_14_volume.shape

(15,)

In [8]:
calculate_vroc(last_14_volume, 14), data[14, 11], round(calculate_vroc(last_14_volume, 14), 2) == data[14, 11]

(14332.14901690238, 14332.15, True)

# JTL

In [9]:
import pandas as pd

# Smoothed Moving Average (SMMA) function
def smma(data, length):
    """
    Calculate Smoothed Moving Average (SMMA)
    Similar to the ta.sma() in Pine Script but smoothed as per Alligator Indicator.
    
    Parameters:
    data (pd.Series): The input data (like closing prices or hl2).
    length (int): The length of the SMMA window.
    
    Returns:
    pd.Series: SMMA values.
    """
    smma = pd.Series(0, index=data.index)
    smma.iloc[length-1] = data.iloc[:length].mean()  # Initial SMA value
    for i in range(length, len(data)):
        smma.iloc[i] = (smma.iloc[i-1] * (length - 1) + data.iloc[i]) / length
    return smma

# Alligator Indicator function
def alligator_indicator(data, jaw_length=13, teeth_length=8, lips_length=5, jaw_offset=21, teeth_offset=13, lips_offset=1):
    """
    Calculate the Williams Alligator Indicator.
    
    Parameters:
    data (pd.Series): Input price data (like closing prices or hl2).
    jaw_length (int): Length for the Jaw SMMA.
    teeth_length (int): Length for the Teeth SMMA.
    lips_length (int): Length for the Lips SMMA.
    jaw_offset (int): Offset for the Jaw line.
    teeth_offset (int): Offset for the Teeth line.
    lips_offset (int): Offset for the Lips line.
    
    Returns:
    pd.DataFrame: A dataframe with Jaw, Teeth, and Lips values.
    """
    # Calculate SMMA for Jaw, Teeth, and Lips
    jaw = smma(data, jaw_length).shift(jaw_offset)
    teeth = smma(data, teeth_length).shift(teeth_offset)
    lips = smma(data, lips_length).shift(lips_offset)
    
    # Combine into a DataFrame
    alligator_df = pd.DataFrame({
        'Jaw': jaw,
        'Teeth': teeth,
        'Lips': lips
    })
    
    return alligator_df

# Example input data (hl2 = (high + low) / 2)
example_data = pd.Series([1.135, 1.128, 1.132, 1.138, 1.142, 1.148, 1.150, 1.155, 1.160, 1.165, 1.170, 1.175, 1.180])

# Calculate the Alligator Indicator
alligator_values = alligator_indicator(example_data)
alligator_values


/var/folders/4_/khzxtjkx373803xbyhfqy0cw0000gn/T/ipykernel_1853/68829705.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.152153846153846' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  smma.iloc[length-1] = data.iloc[:length].mean()  # Initial SMA value
/var/folders/4_/khzxtjkx373803xbyhfqy0cw0000gn/T/ipykernel_1853/68829705.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.141' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  smma.iloc[length-1] = data.iloc[:length].mean()  # Initial SMA value
/var/folders/4_/khzxtjkx373803xbyhfqy0cw0000gn/T/ipykernel_1853/68829705.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.1349999999999998' has dtype i

,Jaw,Teeth,Lips
0,NaN,NaN,NaN
1,NaN,NaN,0.000000
2,NaN,NaN,0.000000
3,NaN,NaN,0.000000
4,NaN,NaN,0.000000
5,NaN,NaN,1.135000
6,NaN,NaN,1.137600
7,NaN,NaN,1.140080
8,NaN,NaN,1.143064
9,NaN,NaN,1.146451


In [10]:
temp = []
for i in range(100):
    hl2 = sum(data[i, 2:4])/2
    temp.append(hl2)

In [11]:
alligator_indicator(pd.Series(temp))

/var/folders/4_/khzxtjkx373803xbyhfqy0cw0000gn/T/ipykernel_1853/68829705.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '128.85346153846154' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  smma.iloc[length-1] = data.iloc[:length].mean()  # Initial SMA value
/var/folders/4_/khzxtjkx373803xbyhfqy0cw0000gn/T/ipykernel_1853/68829705.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '128.37437500000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  smma.iloc[length-1] = data.iloc[:length].mean()  # Initial SMA value
/var/folders/4_/khzxtjkx373803xbyhfqy0cw0000gn/T/ipykernel_1853/68829705.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '128.339' has dtyp

,Jaw,Teeth,Lips
0,NaN,NaN,NaN
1,NaN,NaN,0.000000
2,NaN,NaN,0.000000
3,NaN,NaN,0.000000
4,NaN,NaN,0.000000
...,...,...,...
95,131.628811,131.364438,130.959340
96,131.603517,131.331383,130.992472
97,131.580170,131.301836,131.048978
98,131.550926,131.280356,131.090182


In [12]:
data[99, 5:8]

array([131.52, 131.23, 131.13], dtype=object)

In [13]:
sum(data[1, 2:4])/2, (data[1, 2]+data[1, 3])/2, temp

(128.36,
 128.36,
 [128.315,
  128.36,
  128.345,
  128.325,
  128.35000000000002,
  128.37,
  128.45,
  128.48000000000002,
  128.815,
  129.53,
  129.95999999999998,
  129.895,
  129.9,
  130.57,
  131.28,
  132.03,
  133.14499999999998,
  133.905,
  134.68,
  134.675,
  135.075,
  134.18,
  133.56,
  132.995,
  133.19,
  132.755,
  132.445,
  132.57999999999998,
  132.305,
  132.01,
  132.015,
  132.16500000000002,
  132.035,
  132.235,
  132.24,
  132.36,
  132.75,
  133.085,
  132.64,
  132.66500000000002,
  132.61,
  132.48,
  132.315,
  132.275,
  132.20999999999998,
  132.20999999999998,
  132.05,
  131.9,
  131.865,
  131.8,
  131.87,
  131.835,
  132.04500000000002,
  132.09,
  131.81,
  131.845,
  131.77499999999998,
  131.77499999999998,
  131.77499999999998,
  131.74,
  131.55,
  131.495,
  131.345,
  131.38,
  131.45,
  131.44,
  131.45,
  131.66,
  131.65,
  131.68,
  131.675,
  131.6,
  131.39,
  131.235,
  131.26999999999998,
  131.3,
  131.3,
  131.2,
  131.13,
  131.

In [14]:
import numpy as np
import pandas as pd

# Smoothed Moving Average (SMMA) function using NumPy
def smma(data, length):
    """
    Calculate the Smoothed Moving Average (SMMA) using NumPy.
    
    Parameters:
    data (pd.Series or np.ndarray): The input data (like closing prices or hl2).
    length (int): The length of the SMMA window.
    
    Returns:
    np.ndarray: SMMA values as a NumPy array.
    """
    smma = np.zeros_like(data)
    smma[length - 1] = np.mean(data[:length])  # Initial SMA value
    for i in range(length, len(data)):
        smma[i] = (smma[i - 1] * (length - 1) + data[i]) / length
    return smma

# Alligator Indicator function
def alligator_indicator(data, jaw_length=13, teeth_length=8, lips_length=5, jaw_offset=8, teeth_offset=5, lips_offset=3):
    """
    Calculate the Williams Alligator Indicator using NumPy for efficiency.
    
    Parameters:
    data (pd.Series or np.ndarray): Input price data (like hl2).
    jaw_length (int): Length for the Jaw SMMA.
    teeth_length (int): Length for the Teeth SMMA.
    lips_length (int): Length for the Lips SMMA.
    jaw_offset (int): Offset for the Jaw line.
    teeth_offset (int): Offset for the Teeth line.
    lips_offset (int): Offset for the Lips line.
    
    Returns:
    pd.DataFrame: A dataframe with Jaw, Teeth, and Lips values.
    """
    data = np.array(data)  # Ensure the data is in NumPy array format
    
    # Calculate SMMA for Jaw, Teeth, and Lips
    jaw = smma(data, jaw_length)
    teeth = smma(data, teeth_length)
    lips = smma(data, lips_length)
    
    # Apply the offsets using NumPy's shift mechanism
    jaw_shifted = np.roll(jaw, jaw_offset)
    teeth_shifted = np.roll(teeth, teeth_offset)
    lips_shifted = np.roll(lips, lips_offset)
    
    # For the rolled values, set the first few entries as NaN (due to shifting)
    jaw_shifted[:jaw_offset] = np.nan
    teeth_shifted[:teeth_offset] = np.nan
    lips_shifted[:lips_offset] = np.nan
    
    # Combine into a DataFrame
    alligator_df = pd.DataFrame({
        'Jaw': jaw_shifted,
        'Teeth': teeth_shifted,
        'Lips': lips_shifted
    })
    
    return alligator_df

# Example input data (hl2 = (high + low) / 2)
example_data = pd.Series([1.135, 1.128, 1.132, 1.138, 1.142, 1.148, 1.150, 1.155, 1.160, 1.165, 1.170, 1.175, 1.180])

# Calculate the Alligator Indicator
alligator_values = alligator_indicator(example_data)
alligator_values


,Jaw,Teeth,Lips
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,0.000000
4,NaN,NaN,0.000000
5,NaN,0.000,0.000000
6,NaN,0.000,0.000000
7,NaN,0.000,1.135000
8,0.0,0.000,1.137600
9,0.0,0.000,1.140080


In [15]:
alligator_indicator(temp)

,Jaw,Teeth,Lips
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,0.000000
4,NaN,NaN,0.000000
...,...,...,...
95,131.299791,131.073715,130.980219
96,131.260191,131.071375,130.959175
97,131.229023,131.053078,130.959340
98,131.210636,131.030819,130.992472


In [16]:
data[0]

array(['Wed Sep 25 2024 10:57:00 GMT+0530 (India Standard Time)', 128.35,
       128.36, 128.27, 128.34, 127.57, 127.87, 128.47, 128.42, 128.54,
       8697.0, -68.27], dtype=object)

In [17]:
def SMA(data, period=14):
    """
    Calculate Simple Moving Average (SMA) using NumPy.
    
    Parameters:
    data (np.array): A NumPy array or list of price data (e.g., closing prices).
    period (int): The number of periods to calculate the SMA. Default is 14.
    
    Returns:
    np.array: The Simple Moving Average (SMA) values.
    """
    # Check if the length of data is less than the period
    if len(data) < period:
        return np.array([])  # Not enough data points to compute SMA
    
    # Create an empty array to store SMA values
    sma = np.full(len(data), np.nan)
    
    # Calculate the SMA manually by looping through data points
    for i in range(period - 1, len(data)):
        sma[i] = np.mean(data[i - period + 1:i + 1])
    
    return sma

In [18]:
data[0], (SMA(data[0, 4:5]))

(array(['Wed Sep 25 2024 10:57:00 GMT+0530 (India Standard Time)', 128.35,
        128.36, 128.27, 128.34, 127.57, 127.87, 128.47, 128.42, 128.54,
        8697.0, -68.27], dtype=object),
 array([], dtype=float64))

In [22]:
def sma(source, length):
    cumsum = np.cumsum(np.insert(source, 0, 0))
    # Calculate SMA using cumulative sum and slicing
    sma_values = (cumsum[length:] - cumsum[:-length]) / length
    # Pad the beginning of the array to match the input size
    sma_values = np.concatenate((np.full(length - 1, np.nan), sma_values))
    return sma_values

In [63]:
round(sma(data[:10, 4], 5)[-1], 2)

128.87

In [61]:
data[9, 8]

128.87